# 임포트


In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

# 데이터 로드

In [2]:
class CustomFaceDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png'))])
        self.transform = transform
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_file)
        # 이미지 읽기 및 RGB 변환
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"이미지를 찾을 수 없습니다: {image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 0~1 범위의 실수형 텐서 변환
        image = image.astype(np.float32) / 255.0

        # 어노테이션 파일 경로 (이미지와 같은 이름의 .txt 파일)
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(self.label_dir, label_file)
        boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    # 각 라인은 "class_id center_x center_y width height"
                    parts = line.strip().split()
                    if len(parts) == 5:
                        cls, cx, cy, bw, bh = map(float, parts)
                        boxes.append([int(cls), cx, cy, bw, bh])
        else:
            # 어노테이션이 없는 경우 빈 배열 반환
            boxes = []
        
        # 이미지 shape: [H, W, C] -> tensor: [C, H, W]
        image_tensor = torch.tensor(image.transpose(2, 0, 1), dtype=torch.float32)
        sample = {'image': image_tensor, 'boxes': torch.tensor(boxes, dtype=torch.float32)}
        return sample
